In [ ]:
'''
This is the methods notebook for  SKaWT-Q70* Datasets.

Copyright (C) 2021 - Trupti Gore.

License: GPL-3. 
'''



In [1]:
# import necessary modules
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import skimage.filters as filters
import skimage.color as color
from PIL import Image
import pandas as pd
from skimage.util import img_as_float
from skimage.color import label2rgb, rgb2gray,gray2rgb
from skimage import segmentation,morphology,feature,measure,filters
from scipy import ndimage as ndi
import glob
from natsort import natsorted 
import os
import glob
import re
import random

In [2]:
# import function file
%run functions-Method-ska.ipynb
random.seed(3)

In [3]:
# path to the local folder for the notebooks
BASE_DIR="/Users/Trupti/07-ImageAnalysis/" 
#path to the image folder

FOLDER="SKA"
IMAGE_DIR=BASE_DIR + "data/CMovie/" + FOLDER + "/"
CSV_DIR = BASE_DIR + "Kinetochore-Particle-Tracker/AnalysisResults/CMovie/" + FOLDER +"/" + "csvs/"
GRAPHS_DIR= BASE_DIR + "Kinetochore-Particle-Tracker/GraphAnalysis/AstrinSka/"



#### A List for the Images Folders

In [4]:
wt=[]
ctrl=[]
wt_path = IMAGE_DIR + 'wt'
ctrl_path=IMAGE_DIR + 'control'

for folder in glob.glob(wt_path + '/*'):
    wt.append(wt_path + '/' + folder.split('\\')[-1].split('.')[0])
for folder in glob.glob(ctrl_path + '/*'):
    ctrl.append(ctrl_path + '/' + folder.split('\\')[-1].split('.')[0])     


In [5]:
# For wt

for i in range(len(wt)):
    
    df_GFP_ska,df_CENP_B_ds_red=method_cenpb(wt[i],FOLDER)
    
    df=pd.merge(df_GFP_ska, df_CENP_B_ds_red,  how='outer',on=['label','area']
                                                       ,suffixes=('_GFP_ska', '_CENP_B_ds_red'))
    
    directory=wt[i].split('/')[-1].split('.')[0] # to create a folder per experiment to save csvs
    path = CSV_DIR + 'wt/' + directory
    
    try:
        os.makedirs(path)
    except FileExistsError:
    # directory already exists
        pass
    #pd.concat([df1,df_cdki,df_dmso],ignore_index=True)
    check_nan_in_df = df.isnull().values.any()
    if check_nan_in_df >0:
        df=df.dropna()
        print("NaNs removed from %s " %( directory))
    
    df.to_csv(path + '/' + 'particles'+'.csv') 
   

NaNs removed from EXPT-2021-005-SET5_GFP-SKA3WT_CENPBdsRed_MG132_MITOSIS1_R3D_D3D 
NaNs removed from EXPT-2021-005-SET6_GFP-SKA3WT_CENPBdsRed_MG132_MITOSIS3_R3D_D3D 


In [6]:
# For control

for i in range(len(ctrl)):
    
    df_GFP_ska,df_CENP_B_ds_red=method_cenpb(ctrl[i],FOLDER) 
    
    df=pd.merge(df_GFP_ska, df_CENP_B_ds_red,  how='outer',on=['label','area']
                                                       ,suffixes=('_GFP_ska', '_CENP_B_ds_red'))
    
    directory=ctrl[i].split('/')[-1].split('.')[0] # to create a folder per experiment to save csvs
    path = CSV_DIR + 'ctrl/' + directory
    try:
        os.makedirs(path)
    except FileExistsError:
    # directory already exists
        pass
    #pd.concat([df1,df_cdki,df_dmso],ignore_index=True)
    check_nan_in_df = df.isnull().values.any()
    if check_nan_in_df >0:
        df=df.dropna()
        print("NaNs removed from %s " %( directory))
    
    df.to_csv(path + '/' + 'particles'+'.csv') 
   

NaNs removed from EXPT-2021-005-SET5_GFP-SKA3Q70K_CENPBdsRed_MG132_MITOSIS3_R3D_D3D 
NaNs removed from EXPT-2021-005-SET6_GFP-SKA3Q70K_CENPBdsRed_MG132_MITOSIS4_R3D_D3D 


### Generate csv files for all cells

In [7]:

# The paths are for the generated csv files
wt_csv=[]
ctrl_csv=[]
wt_path = CSV_DIR + 'wt'
ctrl_path=CSV_DIR + 'ctrl'

for folder in natsorted(glob.glob(wt_path + '/*')):
    wt_csv.append(wt_path + '/' + folder.split('\\')[-1].split('.')[0])
    
for folder in natsorted(glob.glob(ctrl_path + '/*')):
    ctrl_csv.append(ctrl_path + '/' + folder.split('\\')[-1].split('.')[0])  
  

In [8]:
# read wildtype csvs to a dataframe
cell_count=1
path = GRAPHS_DIR + FOLDER
try:
    os.makedirs(path)
except FileExistsError:
    # directory already exists
    pass
df_graph_wt=pd.DataFrame()
for i in range(len(wt_csv)):
    df_wt=pd.read_csv(wt_csv[i]+'/'+'Particles.csv')
    df_wt['CENP_B_ds_red@kt/CENP_B_ds_red@cyto']= df_wt['mean_intensity_CENP_B_ds_red']/df_wt['CENP_B_ds_red_cyto']
    df_wt['GFP_ska@kt/GFP_ska@cyto']= df_wt['mean_intensity_GFP_ska']/df_wt['GFP_ska_cyto']
    df_wt['CENP_B_ds_red@kt/CENP_B_ds_red@cyto:GFP_ska@kt/GFP_ska@cyto']= df_wt['CENP_B_ds_red@kt/CENP_B_ds_red@cyto'] / df_wt['GFP_ska@kt/GFP_ska@cyto']
    df_wt['condition']='WildType'
    df_wt['cell']=FOLDER+"_"+'WildT_cell_'+str(cell_count)
    expt_name=re.sub(r'^.+/([^/]+)$', r'\1', wt_csv[i])
    df_wt['Experiment']= expt_name
    fname=FOLDER+'wt_'+ str(i)
    df_wt_final=outliers_ska(df_wt)
    df_wt_final.to_csv(GRAPHS_DIR + FOLDER+'/' + fname +'.csv') 
    df_graph_wt=pd.concat([df_graph_wt,df_wt_final],ignore_index=True)    
    cell_count+=1
df_graph_wt = df_graph_wt.drop(columns=["Unnamed: 0"], axis=1)    

df_graph_wt.to_csv(GRAPHS_DIR + FOLDER+'/' +FOLDER+ "_" +'wt_ska_graphs'+'.csv') 



In [9]:

# read Q70K
cell_count=1
path = GRAPHS_DIR + FOLDER
try:
    os.makedirs(path)
except FileExistsError:
    # directory already exists
    pass
df_graph_ctrl=pd.DataFrame()
for i in range(len(ctrl_csv)):
    df_ctrl=pd.read_csv(ctrl_csv[i]+'/'+'Particles.csv')
    df_ctrl['CENP_B_ds_red@kt/CENP_B_ds_red@cyto']= df_ctrl['mean_intensity_CENP_B_ds_red']/df_ctrl['CENP_B_ds_red_cyto']
    df_ctrl['GFP_ska@kt/GFP_ska@cyto']= df_ctrl['mean_intensity_GFP_ska']/df_ctrl['GFP_ska_cyto']
    df_ctrl['CENP_B_ds_red@kt/CENP_B_ds_red@cyto:GFP_ska@kt/GFP_ska@cyto']= df_ctrl['CENP_B_ds_red@kt/CENP_B_ds_red@cyto'] / df_ctrl['GFP_ska@kt/GFP_ska@cyto']
    df_ctrl['condition']='Q70K'
    df_ctrl['cell']=FOLDER+"_"+'Q70K_cell_'+str(cell_count)
    expt_name=re.sub(r'^.+/([^/]+)$', r'\1', ctrl_csv[i])
    df_ctrl['Experiment']= expt_name
    fname=FOLDER+'ctrl_'+str(i)
    df_ctrl_final=outliers_ska(df_ctrl)
    df_ctrl_final.to_csv(GRAPHS_DIR + FOLDER+'/' + fname +'.csv') 
    df_graph_ctrl=pd.concat([df_graph_ctrl,df_ctrl_final],ignore_index=True)  
    cell_count+=1
    
df_graph_ctrl = df_graph_ctrl.drop(columns=["Unnamed: 0"], axis=1)    

df_graph_ctrl.to_csv(GRAPHS_DIR + FOLDER+'/' +FOLDER+ "_" + 'ctrl_ska_graphs'+'.csv') 


In [10]:
df_graph=pd.DataFrame()
df_graph=pd.concat([df_graph,df_graph_wt,df_graph_ctrl],ignore_index=True)
print(len(df_graph_wt))
print(len(df_graph_ctrl))
print(len(df_graph))
df_graph.head()
df_graph.to_csv(GRAPHS_DIR + FOLDER+'/' +FOLDER+ "_" + 'ska_wt_combined_graphs'+'.csv') 
#df_graph.head()

1712
1870
3582
